In [3]:
import requests
import pandas as pd
import json
from dateutil import parser #convert string to data time
#requests is a library for sending HTTP requests in Python 

In [4]:
API_KEY = "17e7a04426b523ef4eedec284ec3be58-b816384cb30003eefbb6af11ae99dbef"
ACCOUNT_ID = "101-002-26761156-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"
#This is our unique key account id and then we have the url

In [5]:
session = requests.Session()
#stores parameters like cookies and HTTP headers

In [6]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [7]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)
#what exactly we are requesting

In [8]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"
#save url as formatted string taken from postman

In [9]:
response = session.get(url, params=None, data=None, headers=None)
#.get method GET is used to request data from a specified resource.
#the query string (name/value pairs) is sent in the URL of a GET request

In [10]:
response.status_code
#tells us if any errors happened or it requested successfully

200

In [11]:
data = response.json()
#returns back object with all info contains all instruments 
#returns back a dictionary with keys and values

In [16]:
instruments_list = data['instruments']
#saves each different instrument in my instrument list

In [13]:
len(instruments_list)

123

In [14]:
instruments_list[0].keys() #what info we have releated to each object

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [15]:
key_i = ['name', 'type', 'displayName', 'pipLocation',
         'displayPrecision', 'tradeUnitsPrecision', 'marginRate']
#i only want these values from whatever the object returned back to us 

In [20]:
instruments_dict = {}
for i in instruments_list: #loop through entire instrument list
    key = i['name'] #key is equal to instrument name 
    instruments_dict[key] = { k: i[k] for k in key_i } 
    #k would be name, type etc line 15


In [21]:
instruments_dict['USD_CAD']     
#the key would be the name of the pair so USD_CAD and by using that as the key
#i can look at all the info related to USD_CAD for example.

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [22]:
# -4 -> 0.0001 we want pip location to be integer 
pow(10, -4)

0.0001

In [27]:
#where I want to save it 
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))
    #write dictionary we just created into json format 

In [28]:
#pair name = USD_CAD for ex, granularirt is 1 hour default 
#doing same thing as before since we are making a call to the API to get data
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()

    #if successful status return code I want to make sure i get back a candles key back
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data


def get_candles_df(data):
    #if data is empty return empty dataframe 
    if len(data) == 0:
        return pd.DataFrame()
        
    #will be used for column names in DataFrame
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c'] #open,high,low and close key
    #mid_c mid_o bid_h for example

    final_data = []
    
    #visit each candle in the data we passed in
    for candle in data:
        #if incomplete candle is returned we dont want it 
        if candle['complete'] == False:
            continue
        new_dict = {} # new object which will contain out keys from candle data
        # we want time and volume 
        new_dict['time'] = parser.parse(candle['time'])
        # we are given a string for time so convert into date time type 
        new_dict['volume'] = candle['volume']

        #constuct keys like mid_o etc and add data from candle data
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict) #append new dictionary list to final data
        # we created eariler 

    #uses pandas method "dataframe.fron_dict to create dataframe from the list
    #of dictionaries final_data. each dictionary becomes a row in datafrmae
    #and keys becomes column names
    df = pd.DataFrame.from_dict(final_data)
    
    return df
    
#saves dataframe in pickle file 
#pair_name = USD_CAD for ex, default # of candles to fetch is 10 and time of candles
#set default to 1 hour
def create_data_file(pair_name, count=10, granularity="H1"):
    #call function to get candlestick data 
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return #failed attempt to get data return back faliure object
        
    if len(data) == 0:
        print("No candles", pair_name) # if empty return back this

    #convert candle data into dataframe
    candles_df = get_candles_df(data)
    #saves dataframe to pickle file in specfic location
    #filename is based on currency pair name
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    #prints out info about what is in dataframe 
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")
    

In [29]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [30]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 9 candles, 2023-11-13 14:00:00+00:00 2023-11-14 22:00:00+00:00


In [31]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [32]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4000 candles, 2023-03-27 12:00:00+00:00 2023-11-15 04:00:00+00:00
EUR_USD H4 4000 candles, 2021-04-23 09:00:00+00:00 2023-11-14 22:00:00+00:00
EUR_GBP H1 4000 candles, 2023-03-27 12:00:00+00:00 2023-11-15 04:00:00+00:00
EUR_GBP H4 4000 candles, 2021-04-23 09:00:00+00:00 2023-11-14 22:00:00+00:00
EUR_JPY H1 4000 candles, 2023-03-27 12:00:00+00:00 2023-11-15 04:00:00+00:00
EUR_JPY H4 4000 candles, 2021-04-23 01:00:00+00:00 2023-11-14 22:00:00+00:00
EUR_CHF H1 4000 candles, 2023-03-27 12:00:00+00:00 2023-11-15 04:00:00+00:00
EUR_CHF H4 4000 candles, 2021-04-23 09:00:00+00:00 2023-11-14 22:00:00+00:00
EUR_NZD H1 4000 candles, 2023-03-27 11:00:00+00:00 2023-11-15 04:00:00+00:00
EUR_NZD H4 4000 candles, 2021-04-23 09:00:00+00:00 2023-11-14 22:00:00+00:00
EUR_CAD H1 4000 candles, 2023-03-27 12:00:00+00:00 2023-11-15 04:00:00+00:00
EUR_CAD H4 4000 candles, 2021-04-23 09:00:00+00:00 2023-11-14 22:00:00+00:00
EUR_AUD H1 4000 candles, 2023-03-27 11:00:00+00:00 2023-11-15 04:00:00+00:00